Check power spectrum with different neutrino mass

Fixing Omega_m (ensures that k<0.02 does not change in amplitude)

Comparision with CAMB

# Imports

In [1]:
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import sys
sys.path.append('./')
# Reset to default Matplotlib settings
plt.rcdefaults()
from scripts import neutrino_test as nt


# Compute power spectra

Here we compute the power spectra for different neutrino masses using both CAMB and CCL.
We will call the functions defined in scripts/neutrino_test.py to compute the power spectra.
These functions are (note: the ouptu is a tuple: k, P(k)):
- `ccl_power_spectrum`: Computes the power spectrum using CCL.
- `camb_power_spectrum`: Computes the power spectrum using CAMB.

In this exercise we want to compare spectra with and without neutrinos obtained from both libraries, we will call the functions with and without the `mnu` parameter.
Additionally, we also want to investigate how the power spectra change when fixing either $\sigma_8$ or $A_s$.
Since we defined our `CCL` and `CAMB` wrappers to work in the `sigma8` or `As` mode, we can call the functions with the `mode` parameter set to either `sigma8` or `As`.


## Preliminaries


In [2]:
# First we define our k range
# Note that these values are set by default in the `neutrino_test.py` script
# but, it is a good practice to define them here as well so the user is aware of them
min_k = 1e-4
max_k = 2
num_k = 500

# Next, we define two values of neutrino masses mnu to compare
mnu_values = [0.0, 0.3]

# As stated in the description, here we also define both modes to compare
modes = ["sigma8", "As"]

# Lastly, let us define a list of libraries to compare, we will use them in the next cell
libraries = ["camb", "ccl"]

# in cases when there are serveral configurations, it is good to define a dictionary to store the results
pk_dict = {}

# Define the mapping of libraries to their respective functions
library_functions = {
    "camb": nt.camb_power_spectrum,
    "ccl": nt.ccl_power_spectrum
}

In [4]:
%%time
# Iterate over libraries, modes, and neutrino masses and store the power spectra in the dictionary
for library in libraries:
    pk_dict[library] = {}
    for mode in modes:
        pk_dict[library][mode] = {}
        for mnu in mnu_values:
            # Call the appropriate function
            power_spectrum_function = library_functions[library]
            k, pk = power_spectrum_function(mode=mode, mnu=mnu, min_k=min_k, max_k=max_k, num_k=num_k)
            pk_dict[library][mode][f"mnu={mnu}"] = {"k": k, "pk": pk}


sigma8: [0.8225]
sigma8: [0.8225]
sigma8: [0.51322095]
sigma8: [0.47007233]
CPU times: user 3min 28s, sys: 4.6 s, total: 3min 33s
Wall time: 33.8 s


In [5]:
# Check the keys in the dictionary so you can access the power spectra and plot them
print(f"Highest level keys: {pk_dict.keys()}")
print(f"Second level keys: {pk_dict['camb'].keys()}")
print(f"Third level keys: {pk_dict['camb']['sigma8'].keys()}")
print(f"Fourth level keys: {pk_dict['camb']['sigma8']['mnu=0.0'].keys()}")


Highest level keys: dict_keys(['camb', 'ccl'])
Second level keys: dict_keys(['sigma8', 'As'])
Third level keys: dict_keys(['mnu=0.0', 'mnu=0.3'])
Fourth level keys: dict_keys(['k', 'pk'])


K, now that we have computed the power spectra, we can also calcualte the ratios of the power spectra with and without neutrinos.
We will calculate the ratios for both libraries and both modes.

In [6]:
# Define dictionaries to store the ratios
pk_ratios = {}

# Iterate over libraries and modes
for library in libraries:
    pk_ratios[library] = {}
    for mode in modes:
        pk_ratios[library][mode] = {}
        for mnu in mnu_values:
            # Compute the ratio
            pk_ratio = pk_dict[library][mode][f"mnu={mnu}"]["pk"] / pk_dict[library][mode]["mnu=0.0"]["pk"]
            pk_ratios[library][mode][f"mnu={mnu}"] = pk_ratio


In [9]:
# Again, check the keys in the dictionary so you can access the power spectra ratios and plot them
print(f"Highest level keys: {pk_ratios.keys()}")
print(f"Second level keys: {pk_ratios['camb'].keys()}")
print(f"Third level keys: {pk_ratios['camb']['sigma8'].keys()}")


Highest level keys: dict_keys(['camb', 'ccl'])
Second level keys: dict_keys(['sigma8', 'As'])
Third level keys: dict_keys(['mnu=0.0', 'mnu=0.3'])


 # Plotting the power spectra and their ratios

Now that we computed the power spectra and their ratios (for our desired configurations), we can plot them.


In [12]:
# Define the colors, line styles, and line widths for the plots

# First let us fetch the values of mnu we used and create string keys dynamically
mnu_labels = {mnu: f"mnu={mnu}" for mnu in mnu_values}

# Define the colors dictionary dynamically using mnu_values and multiple color schemes
colors = {
    library: {
        mode: {mnu_labels[mnu]: color for mnu, color in zip(mnu_values, ["darkorange", "teal"] if mode == "sigma8" else ["orangered", "darkslategrey"])}
        for mode in ["sigma8", "As"]
    }
    for library in ["camb", "ccl"]
}

# Print the resulting dictionary to verify
print(colors)

{'camb': {'sigma8': {'mnu=0.0': 'darkorange', 'mnu=0.3': 'teal'}, 'As': {'mnu=0.0': 'orangered', 'mnu=0.3': 'darkslategrey'}}, 'ccl': {'sigma8': {'mnu=0.0': 'darkorange', 'mnu=0.3': 'teal'}, 'As': {'mnu=0.0': 'orangered', 'mnu=0.3': 'darkslategrey'}}}


In [ ]:
# Create a figure with 6 subplots (2 rows, 3 columns)
fig, axes = plt.subplots(2, 3, figsize=(18, 10), sharex=True)

# Iterate over rows (first row for 'sigma8', second row for 'As')
modes = ["sigma8", "As"]
for row, mode in enumerate(modes):
    # Iterate over columns (CAMB & CCL spectra, ratio CAMB/CCL no neutrino, ratio CAMB/CCL neutrino)
    for col, scenario in enumerate(["spectra", "ratio_no_nu", "ratio_with_nu"]):
        ax = axes[row, col]

        if scenario == "spectra":
            # Plot CAMB and CCL spectra for mnu=0 and mnu=0.3
            for mnu in ["mnu=0", "mnu=0.3"]:
                mnu_label = {
                    "mnu=0": "0",
                    "mnu=0.3": "0.3"
                }
                ax.loglog(
                    pk["camb"][mode][mnu]["k"],
                    pk["camb"][mode][mnu]["pk"],
                    color=colors["camb"][mode][mnu],
                    ls=line_styles["camb"][mode][mnu],
                    linewidth=line_widths["camb"][mode][mnu],
                    label=f"CAMB $m_{{\\nu}} = {mnu_label[mnu]}$"
                )
                ax.loglog(
                    pk["ccl"][mode][mnu]["k"],
                    pk["ccl"][mode][mnu]["pk"],
                    color=colors["ccl"][mode][mnu],
                    ls=line_styles["ccl"][mode][mnu],
                    linewidth=line_widths["ccl"][mode][mnu],
                    label=f"CCL $m_{{\\nu}} = {mnu_label[mnu]}$"
                )
            mode_label = {
                "sigma8": "$\\sigma_8$",
                "As": "$A_s$"
            }
            ax.set_ylabel(r"$P(k)$", fontsize=14)
            ax.set_title(f"$P(k)$ for {mode_label[mode]} fixed", fontsize=16)
            ax.legend()

        elif scenario == "ratio_no_nu":
            # Plot ratio of CAMB no neutrino to CCL no neutrino
            ax.semilogx(
                pk["ccl"][mode]["mnu=0"]["k"],
                pk["camb"][mode]["mnu=0"]["pk"] / pk["ccl"][mode]["mnu=0"]["pk"] - 1,
                color="yellowgreen",
                ls="-",
                linewidth=3,
                label="CAMB $m_{\\nu} = 0$ / CCL mnu=0"
            )
            ax.set_ylabel(r"$P(k)$ Ratio", fontsize=14)
            ax.set_title(f"Ratio (No neutrinos): {mode}", fontsize=16)

        elif scenario == "ratio_with_nu":
            # Plot ratio of CAMB with neutrino to CCL with neutrino
            ax.semilogx(
                pk["ccl"][mode]["mnu=0.3"]["k"],
                pk["camb"][mode]["mnu=0.3"]["pk"] / pk["ccl"][mode]["mnu=0.3"]["pk"] - 1,
                color="cadetblue",
                ls="--",
                linewidth=3,
                label="CAMB mnu=0.3 / CCL mnu=0.3"
            )
            ax.set_ylabel(r"$P(k)$ Ratio", fontsize=14)
            ax.set_title(f"$P(k)$ Ratio (with neutrinos): {mode.upper()}", fontsize=16)

        # Common x-axis label for all subplots
        ax.set_xlabel(r"$k\, [h\, \mathrm{Mpc}^{-1}]$", fontsize=14)

        # Add legend and grid
        #ax.legend()
        ax.grid(False)

# Adjust layout and show plot
#plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10), sharex=True)

# Iterate over rows (first row for 'sigma8', second row for 'As')
modes = ["sigma8", "As"]
for row, mode in enumerate(modes):
    # First column for spectra, second column for combined ratios
    for col, scenario in enumerate(["spectra", "ratios"]):
        ax = axes[row, col]

        if scenario == "spectra":
            # Plot CAMB and CCL spectra for mnu=0 and mnu=0.3
            for mnu in ["mnu=0", "mnu=0.3"]:
                mnu_label = {
                    "mnu=0": "0",
                    "mnu=0.3": "0.3"
                }
                ax.loglog(
                    pk["camb"][mode][mnu]["k"],
                    pk["camb"][mode][mnu]["pk"],
                    color=colors["camb"][mode][mnu],
                    ls=line_styles["camb"][mode][mnu],
                    linewidth=line_widths["camb"][mode][mnu],
                    label=f"CAMB $m_{{\\nu}} = {mnu_label[mnu]}$"
                )
                ax.loglog(
                    pk["ccl"][mode][mnu]["k"],
                    pk["ccl"][mode][mnu]["pk"],
                    color=colors["ccl"][mode][mnu],
                    ls=line_styles["ccl"][mode][mnu],
                    linewidth=line_widths["ccl"][mode][mnu],
                    label=f"CCL $m_{{\\nu}} = {mnu_label[mnu]}$"
                )
            mode_label = {
                "sigma8": "$\\sigma_8$",
                "As": "$A_s$"
            }
            ax.set_ylabel(r"$P(k)$", fontsize=14)
            ax.set_title(f"$P(k)$ for {mode_label[mode]} fixed", fontsize=16)
            ax.legend()

        elif scenario == "ratios":
            # Plot both ratios in the same subplot
            ax.semilogx(
                pk["ccl"][mode]["mnu=0"]["k"],
                pk["camb"][mode]["mnu=0"]["pk"] / pk["ccl"][mode]["mnu=0"]["pk"] - 1,
                color="yellowgreen",
                ls="-",
                linewidth=3,
                label="No Neutrino: CAMB/CCL"
            )
            ax.semilogx(
                pk["ccl"][mode]["mnu=0.3"]["k"],
                pk["camb"][mode]["mnu=0.3"]["pk"] / pk["ccl"][mode]["mnu=0.3"]["pk"] - 1,
                color="cadetblue",
                ls="--",
                linewidth=3,
                label="With Neutrino: CAMB/CCL"
            )
            ax.set_ylabel(r"$P(k)$ Ratio", fontsize=14)
            ax.set_title(f"$P(k)$ Ratios for {mode.upper()}", fontsize=16)
            ax.legend()

        # Common x-axis label for all subplots
        ax.set_xlabel(r"$k\, [h\, \mathrm{Mpc}^{-1}]$", fontsize=14)

        # Add grid and remove if not wanted
        ax.grid(False)

# Adjust layout and show plot
plt.tight_layout()
plt.show()


In [ ]:
# Create figure and gridspec for side-by-side plots with residuals
fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(2, 2, height_ratios=[3, 1], hspace=0.05, wspace=0.3)  # Adjust spacing

modes = ["sigma8", "As"]
mode_labels = {"sigma8": "$\\sigma_8$", "As": "$A_s$"}

for i, mode in enumerate(modes):
    # Main spectra plot
    ax_main = fig.add_subplot(gs[0, i])
    for mnu in ["mnu=0", "mnu=0.3"]:
        mnu_label = {"mnu=0": "0", "mnu=0.3": "0.3"}
        ax_main.loglog(
            pk["camb"][mode][mnu]["k"],
            pk["camb"][mode][mnu]["pk"],
            label=f"CAMB $m_{{\\nu}} = {mnu_label[mnu]}$"
        )
        ax_main.loglog(
            pk["ccl"][mode][mnu]["k"],
            pk["ccl"][mode][mnu]["pk"],
            label=f"CCL $m_{{\\nu}} = {mnu_label[mnu]}$"
        )
    ax_main.set_ylabel(r"$P(k)$", fontsize=14)
    ax_main.set_title(f"$P(k)$ for {mode_labels[mode]} fixed", fontsize=16)
    ax_main.legend()
    ax_main.grid(False)

    # Residuals plot
    ax_residuals = fig.add_subplot(gs[1, i], sharex=ax_main)
    ax_residuals.semilogy(
        pk["ccl"][mode]["mnu=0"]["k"],
        abs(pk["camb"][mode]["mnu=0"]["pk"] / pk["ccl"][mode]["mnu=0"]["pk"] - 1),
        color="yellowgreen",
        ls="-",
        linewidth=2,
        label="No Neutrino: CAMB/CCL"
    )
    ax_residuals.semilogy(
        pk["ccl"][mode]["mnu=0.3"]["k"],
        abs(pk["camb"][mode]["mnu=0.3"]["pk"] / pk["ccl"][mode]["mnu=0.3"]["pk"] - 1),
        color="cadetblue",
        ls="--",
        linewidth=2,
        label="With Neutrino: CAMB/CCL"
    )
    ax_residuals.set_ylabel(r"Residual", fontsize=14)
    ax_residuals.set_xlabel(r"$k\, [h\, \mathrm{Mpc}^{-1}]$", fontsize=14)
    ax_residuals.axhline(0, color="gray", lw=1, ls="--")  # Reference line at zero
    ax_residuals.legend()
    ax_residuals.grid(False)

# Automatically adjust layout
#plt.tight_layout()
plt.show()


In [ ]:
plt.loglog(k_camb, pk2_camb[0,:], color='r', ls = '-',label = 'nu=0.3')
plt.loglog(k_camb, pk_camb[0,:], color='b', ls = '-',label = 'nu=0.')

plt.loglog(camb_k, camb_pk2[0,:], color='r', ls = ':',label = 'nu=0.3')
plt.loglog(camb_k, camb_pk[0,:], color='b', ls = '',label = 'nu=0.')

plt.legend()
plt.ylabel('$PK$')
plt.xlabel(r'$k\, [h \,\rm{Mpc}^{-1}]$')
plt.title('CAMB, fixing sigma8')
plt.grid(False)

In [ ]:
plt.loglog(k_ccl, pk2_ccl, color='r', ls = '-',label = 'nu=0.3')
plt.loglog(k_ccl, pk_ccl, color='b', ls = '-',label = 'nu=0.')
plt.legend()
plt.ylabel('$PK$')
plt.xlabel(r'$k\, [h \,\rm{Mpc}^{-1}]$')
plt.title('CCL, fixing sigma8')

In [ ]:
#can not make CCL and camb match when I fix sigm8

plt.loglog(k, pk_ccl, color='r', ls = '-',label = 'CCL')
plt.loglog(k_camb, pk_camb[0,:], color='b', ls = '-',label = 'CAMB')
plt.title('nu=0 sigma8 fixed')
plt.legend()


In [ ]:
k_camb,pk_camb = Pk_camb(mode = "As")
k_camb,pk2_camb = Pk_camb(mode = "As",mnu = 0.3)

plt.loglog(k_camb, pk2_camb[0,:], color='r', ls = '-',label = 'nu=0.3')
plt.loglog(k_camb, pk_camb[0,:], color='b', ls = '-',label = 'nu=0.')
plt.legend()
plt.ylabel('$PK$')
plt.xlabel(r'$k\, [h \,\rm{Mpc}^{-1}]$')
plt.title('CAMB, fixing As')

In [ ]:
k,pk_ccl = Pk_ccl(mode = "As")
k,pk2_ccl = Pk_ccl(mode = "As", mnu = 0.3)

plt.loglog(k, pk2_ccl, color='r', ls = '-',label = 'nu=0.3')
plt.loglog(k, pk_ccl, color='b', ls = '-',label = 'nu=0.')
plt.legend()
plt.ylabel('$PK$')
plt.xlabel(r'$k\, [h \,\rm{Mpc}^{-1}]$')
plt.title('CCL, fixing As')

In [ ]:
#When fixing As, they look more similar 
#however, the nu=0.3/nu=0 does not match (different model?)

plt.loglog(k, pk_ccl, color='r', ls = '-',label = 'CCL nu=0')
plt.loglog(k_camb, pk_camb[0,:], color='b', ls = '--',label = 'CAMB nu=0')


plt.loglog(k, pk2_ccl, color='C1', ls = '-',label = 'CCL nu=0.3')
plt.loglog(k_camb, pk2_camb[0,:], color='C0', ls = '--',label = 'CAMB nu=0.3')
plt.title('As fixed')
plt.legend()

In [ ]:
#When fixing As, they look more similar 
#however, the nu=0.3/nu=0 does not match (different model?)

plt.semilogx(k, pk_ccl, color='r', ls = '-',label = 'CCL nu=0')
plt.semilogx(k_camb, pk_camb[0,:], color='b', ls = '--',label = 'CAMB nu=0')


plt.semilogx(k, pk2_ccl, color='C1', ls = '-',label = 'CCL nu=0.3')
plt.semilogx(k_camb, pk2_camb[0,:], color='C0', ls = '--',label = 'CAMB nu=0.3')
plt.title('As fixed')
plt.legend()

plt.gca().set_ylim((6000,10000))
plt.gca().set_xlim((6e-3,4e-2))

In [ ]:
plt.semilogx(k_camb, pk2_camb[0,:]/pk_camb[0,:], color='r', ls = '-', label = 'CAMB ')
plt.semilogx(k, pk2_ccl/pk_ccl, color='b', ls = '-',label = 'CCL ')

plt.xlabel(r'$k\, [h \,\rm{Mpc}^{-1}]$')
plt.ylabel(r'$Pk(nu=0.3)/Pk(nu=0.0)$')
plt.legend()
plt.title('As fixed')